In [5]:
import numpy as np
import talib

In [32]:
size = 10
close = np.array([1.0, 16.0, 9.0, 1.0, 1.0, 2.0, 6.0, 9.0, 9.0, 1.0, 6.0, 12.0])
result = talib.RSI(close, 2)
ma = talib.SMA(close, 2)
#print ma
print result

[         nan          nan  68.18181818  39.47368421  39.47368421  50.
  79.09090909  88.83495146  88.83495146  14.87804878  58.28685259
  81.24328198]
